# Warehouse Allocation
Pada sesi ini kita akan menggunakan library python pulp untuk menyelesaikan permasalahan alokasi gudang. Tujuan dari permasalahan ini yaitu menentukan gudang di kota mana saja yang akan dibuka supaya dapat meminimalkan jarak distribusi dari gudang ke customer. Materi ini bersumber dari kursus udemy **Supply Chain Design and Planning with Excel & Python - Haytham Omar.** Pada permasalahan kita tidak menggunakan data kapasitas gudang maupun biaya pembuatan/pembukaan gudang. Oleh karena itu, biaya yang diperlukan untuk membuka gudang diasumsikan sama.

### 1. Mengimpor Library

In [1]:
import numpy as np
from pulp import *
import pandas as pd

### 2. Mengimpor Data
Data yang akan kita gunakan yakni data jarak distribusi dari gudang menuju customer (warehouse_city.xlsx) serta data demand masing-masing customer.

In [2]:
#kumpulkan data
dist = pd.read_excel('warehouse_city.xlsx').set_index('Warehouse')
demand = [10000,20000,33000,9000,60000,2500,35000] #demand masing-masing customer

#melihat tabel dist
dist

,city 1,city 2,city 3,city 4,city 5,city 6,city 7
Warehouse,,,,,,,
city 1,0,2304,2528,950,906,947,1475
city 2,2304,0,2829,1776,2310,1475,2192
city 3,2528,2829,0,3123,1965,2134,1187
city 4,950,1776,3123,0,1699,1149,1926
city 5,906,2310,1965,1699,0,842,915
city 6,947,1475,2134,1149,842,0,922
city 7,1475,2192,1187,1926,915,922,0


### 3. Membuat Variabel
Selanjutnya kita akan membuat parameter dan variabel yang dibutuhkan.

**Parameter:**
- warehouse: kumpulan gudang yang tersedia
- customer: kumpulan pelanggan

**Variabel:**
- flows: variabel bertipe binary yang menunjukkan apakah akan dilakukan distribusi dari gudang menuju customer
- open_w: variable bertipe binary yang menunjukkan apakah suatu gudang akan dibuka atau tidak

In [3]:
#membuat parameter
warehouse = dist.index
customer = dist.columns

#membuat key -> untuk membuat variabel flow
keys = [(w,c) for w in warehouse for c in customer]

#transformasi data jarak dan demand supaya lebih mudah diakses pulp
dist_dict = {(w,c):dist.loc[w,c] for w in warehouse for c in customer}
demand_dict = dict(zip(customer,demand))

#inisiasi variabel
flows = LpVariable.dicts('flows', keys, cat='Binary')
open_w = LpVariable.dicts('open_w', warehouse, cat='Binary')

In [5]:
demand_dict

{'city 1': 10000,
 'city 2': 20000,
 'city 3': 33000,
 'city 4': 9000,
 'city 5': 60000,
 'city 6': 2500,
 'city 7': 35000}

In [11]:
dist_dict

{('city 1', 'city 1'): 0,
 ('city 1', 'city 2'): 2304,
 ('city 1', 'city 3'): 2528,
 ('city 1', 'city 4'): 950,
 ('city 1', 'city 5'): 906,
 ('city 1', 'city 6'): 947,
 ('city 1', 'city 7'): 1475,
 ('city 2', 'city 1'): 2304,
 ('city 2', 'city 2'): 0,
 ('city 2', 'city 3'): 2829,
 ('city 2', 'city 4'): 1776,
 ('city 2', 'city 5'): 2310,
 ('city 2', 'city 6'): 1475,
 ('city 2', 'city 7'): 2192,
 ('city 3', 'city 1'): 2528,
 ('city 3', 'city 2'): 2829,
 ('city 3', 'city 3'): 0,
 ('city 3', 'city 4'): 3123,
 ('city 3', 'city 5'): 1965,
 ('city 3', 'city 6'): 2134,
 ('city 3', 'city 7'): 1187,
 ('city 4', 'city 1'): 950,
 ('city 4', 'city 2'): 1776,
 ('city 4', 'city 3'): 3123,
 ('city 4', 'city 4'): 0,
 ('city 4', 'city 5'): 1699,
 ('city 4', 'city 6'): 1149,
 ('city 4', 'city 7'): 1926,
 ('city 5', 'city 1'): 906,
 ('city 5', 'city 2'): 2310,
 ('city 5', 'city 3'): 1965,
 ('city 5', 'city 4'): 1699,
 ('city 5', 'city 5'): 0,
 ('city 5', 'city 6'): 842,
 ('city 5', 'city 7'): 915,
 ('city

### 4. Menginisiasi Model

In [12]:
#inisiasi model
model = LpProblem('Warehouse_Allocation', LpMinimize)

### 5. Menambahkan Fungsi Tujuan dan Constrain

Fungsi tujuan dari model ini adalah meminimalkan total jarak distribusi dari masing-masing gudang menuju masing-masing customer. Total jarak dihitung berdasarkan perkalian dari demand customer c dengan jarak antara gudang w ke customer c.

Adapun batasan dari model ini yaitu:
1. Pastikan setiap customer mendapatkan pasokan dari salah satu gudang
2. Asumsi jumlah gudang yang akan dibuka sebanyak 3 unit
3. Pastikan jika gudang di kota i akan dibuka, maka unit juga akan dikirimkan pada customer di kota i

In [14]:
#Fungsi tujuan (meminimalkan effort transportasi unit dari warehouse ke customer)
model += lpSum([demand_dict[c]*flows[(w,c)]*dist_dict[(w,c)] for w in warehouse for c in customer])

In [15]:
#Constrain

#Pastikan setiap customer mendapatkan pasokan dari salah satu gudang
for c in customer:
    model += lpSum(flows[(w,c)] for w in warehouse)==1

#Jumlah gudang yang akan dibuka ada 3
model += lpSum(open_w[w] for w in warehouse) == 3

#Pastikan jika gudang ke i akan dibuka, maka unit juga akan dikirimkan pada customer di kota i
for w in warehouse:
    for c in customer:
        model += open_w[w] >= flows[(w,c)]

### 6. Menyelesaikan Model dan Mengecek Hasil Optimasi
Selain mengecek jarak yang ditempuh, kita juga bisa mengecek gudang mana yang akan dibuka serta bagaimana alokasi gudang tersebut ke masing-masing customer

In [20]:
#Selesaikan model -> jika outputnya 1 artinya model telah optimal
model.solve()

1

In [21]:
#cek jarak
model.objective.value()

58481000

In [22]:
#Gudang mana yang akan dibuka?
open_or_not = []

for w in warehouse:
    if open_w[w].varValue == 1:
        status = 'Open'
        open_or_not.append(status)
    else:
        status = 'Not Open'
        open_or_not.append(status)
        
warehouse_status = pd.DataFrame({'Warehouse':warehouse,
                                 'Status':open_or_not}).set_index('Warehouse')

warehouse_status

,Status
Warehouse,
city 1,Not Open
city 2,Open
city 3,Open
city 4,Not Open
city 5,Open
city 6,Not Open
city 7,Not Open


In [23]:
#Bagaimanakah alokasi gudang ke customer?
f = []

for w in warehouse:
    for c in customer:
        f.append(flows[(w,c)].varValue)
        
f_grouped = [f[i:i+7] for i in range(0,len(f),7)]
distribution = pd.DataFrame(f_grouped, columns=customer, index=warehouse)
distribution

#1 artinya dilakukan, 0 artinya tidak

,city 1,city 2,city 3,city 4,city 5,city 6,city 7
Warehouse,,,,,,,
city 1,0,0,0,0,0,0,0
city 2,0,1,0,0,0,0,0
city 3,0,0,1,0,0,0,0
city 4,0,0,0,0,0,0,0
city 5,1,0,0,1,1,1,1
city 6,0,0,0,0,0,0,0
city 7,0,0,0,0,0,0,0


Berdasarkan hasil optimasi di atas gudang yang akan dibuka yakni gudang pada kota 2, 3, dan 5. Gudang pada kota 5 memasok produk paling banyak. Kita juga bisa melakukan simulasi jumlah gudang yang dibuka terhadap total jarak yang ditempuh.

In [26]:
def warehouse_simulation(n):
    model = LpProblem('Warehouse_Simulatiom', LpMinimize)
    #Fungsi tujuan (meminimalkan effort transportasi unit dari warehouse ke customer)
    model += lpSum([demand_dict[c]*flows[(w,c)]*dist_dict[(w,c)] for w in warehouse for c in customer])
    
    #Constrain

    #Pastikan setiap customer mendapatkan pasokan dari salah satu gudang
    for c in customer:
        model += lpSum(flows[(w,c)] for w in warehouse)==1

    #Jumlah gudang yang akan dibuat ada 3
    model += lpSum(open_w[w] for w in warehouse) == n

    #Pastikan jika gudang ke i akan dibuka, maka unit juga akan dikirimkan pada customer di kota i
    for w in warehouse:
        for c in customer:
            model += open_w[w] >= flows[(w,c)]
             
    #Selesaikan model -> jika outputnya 1 artinya model telah optimal
    model.solve()
    return model.objective.value() 

In [27]:
for n in range(1,8):
    print(f"Jumlah gudang:{n} dengan total jarak {warehouse_simulation(n)}")

Jumlah gudang:1 dengan total jarak 169526000
Jumlah gudang:2 dengan total jarak 104681000
Jumlah gudang:3 dengan total jarak 58481000
Jumlah gudang:4 dengan total jarak 26456000
Jumlah gudang:5 dengan total jarak 10655000
Jumlah gudang:6 dengan total jarak 2105000
Jumlah gudang:7 dengan total jarak 0


# Penutup

Oke, ternyata semakin banyak gudang yang dibuka maka total jarak yang dibutuhkan akan semakin berkurang. Oleh karena itu keputusan mengenai berapakah gudang yang akan dibuka juga tergantung pada budget yang tersedia. Demikian tutorial singkat menyelesaikan permasalahan alokasi gudang, apabila terdapat kekurangan saya mohon maaf. Sekian dan terima kasih, selamat mencoba :)